In [16]:
import sys
import os
import importlib

cur=os.getcwd()
paths = [cur := os.path.dirname(cur) for _ in range(3)]
sys.path.insert(0, paths[0])
from Helpers import loadData, sphereMask, saveData, visualizer
importlib.reload(sys.modules['Helpers'])
importlib.reload(sys.modules['Helpers.visualizer'])

import h5py
import matplotlib.pyplot as plt
import numpy as np
from skimage.feature import peak_local_max

In [17]:
# Establish variable conditions
diameter=100
width=1.3
Cutoff=5
MinSep=5

highHist=250
lowHist=10

maxDwTicker=10
mindeldiameter=.0001

maxTicker=10
mindelchi2=1

In [18]:
# Load Data
dataFolder = os.path.join(paths[1],'Data')
oldData = loadData(dataFolder, 'downscale_17.hdf5')
data = loadData(dataFolder, 'convMap_17.hdf5')

# Find peaks
peaks = peak_local_max(data, min_distance=7)

In [19]:
def create_particle_grid(peak_x_positions, peak_y_positions, grid_width, grid_height, 
                        boundary_rect, num_peaks, kernel_padding_size, use_radius):
    """
    Create a local grid centered on each particle and an overlap matrix for Voronoi volumes
    
    Parameters:
    -----------
    peak_x_positions : array-like
        X coordinates of peak positions
    peak_y_positions : array-like
        Y coordinates of peak positions
    grid_width : int
        Width of the grid (x dimension)
    grid_height : int
        Height of the grid (y dimension)
    boundary_rect : array-like
        Rectangle boundaries [y_min, y_max, x_min, x_max]
    num_peaks : int
        Number of peaks/particles
    kernel_padding_size : float
        Size of the kernel padding region
    use_radius : bool
        If True, use absolute distance; if False, use complex distance
    
    Returns:
    --------
    particle_grid : ndarray
        Complex grid (cx + i*cy) centered on each particle
    overlap_map : ndarray
        Image showing which particle (1-indexed) owns each pixel in its Voronoi region
        
    Notes:
    ------
    Creates a local grid for the Voronoi volume of each particle.
    The union of all grids covers the entire image within the boundary rectangle.
    Maximum individual grid size is 2*ceil(kernel_padding_size/2).
    
    Revision history:
    01/16/01 Mark D. Shattuck <mds> calcimg.m
    02/30/06 mds rename pgrid.m return peakGrid and overlap instead of ci
    """
    
    # Handle empty input
    if len(peak_x_positions) == 0:
        particle_grid = np.zeros((grid_width, grid_height), dtype=complex)
        overlap_map = np.zeros((grid_width, grid_height))
        return particle_grid, overlap_map
    
    # Extract boundary coordinates
    y_min = boundary_rect[0]
    y_max = boundary_rect[1]
    x_min = boundary_rect[2]
    x_max = boundary_rect[3]
    
    # Create coordinate meshgrids (1-indexed to match MATLAB convention)
    x_coords, y_coords = np.meshgrid(
        np.arange(1, grid_width + 1), 
        np.arange(1, grid_height + 1), 
        indexing='ij'
    )
    
    # Initialize particle grid with large values
    max_dimension = max(grid_width, grid_height)
    particle_grid = np.ones((grid_width, grid_height), dtype=complex) * max_dimension
    
    # Initialize overlap map (tracks which particle owns each pixel)
    overlap_map = np.zeros((grid_width, grid_height))
    
    # Define kernel range (region around each particle to check)
    half_kernel = int(np.ceil(kernel_padding_size / 2))
    kernel_offset_x = np.arange(-half_kernel, half_kernel + 1)
    kernel_offset_y = kernel_offset_x.copy()
    
    # Process each particle/peak
    for particle_idx in range(num_peaks):
        # Calculate extended kernel range around this particle
        kernel_range_x = np.round(peak_x_positions[particle_idx]) + kernel_offset_x
        kernel_range_y = np.round(peak_y_positions[particle_idx]) + kernel_offset_y
        
        # Clip to boundary rectangle
        valid_x = kernel_range_x[(kernel_range_x <= x_max) & (kernel_range_x >= x_min)].astype(int)
        valid_y = kernel_range_y[(kernel_range_y <= y_max) & (kernel_range_y >= y_min)].astype(int)
        
        # Only process if particle influence is inside boundary
        if valid_x.size > 0 and valid_y.size > 0:
            # Convert to 0-based indices for NumPy
            valid_x_idx = valid_x - 1
            valid_y_idx = valid_y - 1
            
            # Calculate distance from particle to each grid point in the region
            delta_x = x_coords[np.ix_(valid_x_idx, valid_y_idx)] - peak_x_positions[particle_idx]
            delta_y = y_coords[np.ix_(valid_x_idx, valid_y_idx)] - peak_y_positions[particle_idx]
            
            if use_radius:
                # Use absolute distance (Euclidean)
                distance_grid = np.abs(delta_x + 1j * delta_y)
                
                # Find pixels where this particle is closer than current assignment
                closer_mask = particle_grid[np.ix_(valid_x_idx, valid_y_idx)] >= distance_grid
                closer_x, closer_y = np.where(closer_mask)
                
                # Update overlap map and particle grid for closer pixels
                for i, j in zip(closer_x, closer_y):
                    overlap_map[valid_x_idx[i], valid_y_idx[j]] = particle_idx + 1  # 1-indexed
                    particle_grid[valid_x_idx[i], valid_y_idx[j]] = distance_grid[i, j]
                    
            else:
                # Use complex distance (preserves direction)
                distance_grid = delta_x + 1j * delta_y
                
                # Find pixels where this particle is closer than current assignment
                closer_mask = np.abs(particle_grid[np.ix_(valid_x_idx, valid_y_idx)]) >= np.abs(distance_grid)
                closer_x, closer_y = np.where(closer_mask)
                
                # Update overlap map and particle grid for closer pixels
                for i, j in zip(closer_x, closer_y):
                    overlap_map[valid_x_idx[i], valid_y_idx[j]] = particle_idx + 1  # 1-indexed
                    particle_grid[valid_x_idx[i], valid_y_idx[j]] = distance_grid[i, j]
    
    return particle_grid, overlap_map

In [20]:
def create_ideal_particle_image(radial_grid, diameter, width):
    """
    Calculate ideal particle image with smooth edge falloff.
    
    Parameters:
    -----------
    radial_grid : ndarray
        Grid of radial distances from particle center
    diameter : float
        Diameter of the particle
    width : float
        Width parameter controlling edge sharpness.
        2*width represents the distance over which 76% of the intensity falloff occurs
    
    Returns:
    --------
    ideal_particle_image : ndarray
        Image of ideal particle with values from 0 (far from particle) to 1 (center)
        
    Notes:
    ------
    Uses hyperbolic tangent (tanh) for smooth transition from particle to background.
    The transition is centered at diameter/2 and controlled by the width parameter.
    
    Revision history:
    08/04/05 Mark D. Shattuck <mds> ipf.m
    01/30/06 mds added abs(radialGrid)
    04/30/07 mds made w a true measure of width
    """
    # Calculate normalized distance from particle edge
    distance_from_edge = (np.abs(radial_grid) - diameter/2) / width
    
    # Create smooth falloff using tanh: 1 at center, 0 far from particle
    ideal_particle_image = (1 - np.tanh(distance_from_edge)) / 2
    
    return ideal_particle_image

In [21]:
# Setup for ideal particle
kernel_size = 2 * int(diameter/2 + 4*width/2) - 1  # size of ideal particle image
half_kernel_size = (kernel_size - 1) / 2            # (size-1)/2 of ideal particle image
coord_range = np.arange(-half_kernel_size, half_kernel_size + 1)
x_grid, y_grid = np.meshgrid(coord_range, coord_range, indexing='ij')  # ideal particle image grid
radial_grid = np.hypot(x_grid, y_grid)              # radial coordinate

# Extract peak locations
num_peaks = len(peaks)
peak_x_locations = peaks[:, 0]  # Assuming peaks is Nx2 array
peak_y_locations = peaks[:, 1]

# Get image dimensions
grid_width, grid_height = data.shape

# Create local grid centered on each particle and overlap matrix
peak_grids, overlap_map = create_particle_grid(
    peak_x_locations - half_kernel_size, 
    peak_y_locations - half_kernel_size,
    grid_width, 
    grid_height,
    [1, grid_width, 1, grid_height], 
    num_peaks, 
    2 * half_kernel_size + 3, 
    0
)

# Create ideal particle kernels
peak_kernels = create_ideal_particle_image(peak_grids, diameter, width)

# Calculate residuals and chi-squared
residuals = peak_kernels - data  # Calculate difference image
chi_squared = np.sum(residuals**2)  # Calculate Chi-Squared
print(f'Chi-Squared={chi_squared}')

# Save for later optimized comparison
chi_squared_initial = chi_squared
residuals_initial = residuals.copy()

ValueError: too many values to unpack (expected 2)

In [ ]:
def optimize_diameter_width(peak_grids, residuals, diameter, width):
    """
    Calculate one Newton's step toward minimizing residuals^2 over diameter and width.
    
    Parameters:
    -----------
    peak_grids : ndarray
        Grid of distances from particle centers
    residuals : ndarray
        Difference between model and actual image
    diameter : float
        Current particle diameter estimate
    width : float
        Current width parameter estimate
    
    Returns:
    --------
    del_diameter : float
        Change in diameter to minimize residuals
    del_width : float
        Change in width to minimize residuals
        
    Notes:
    ------
    Uses Newton's method with approximate Hessian to find parameter updates.
    
    Revision history:
    09/14/00 Mark D. Shattuck <mds> cidDw.m
    04/30/07 mds changed meaning of width to 1/width
    """
    
    # Create general params
    particle_kern = peak_grids - diameter/2
    inv_width = 1/width
    hessian = np.zeros((2, 2))
    
    # Create starter functions
    tanh_term = np.tanh(particle_kern * inv_width)
    sech2_term = (1 / np.cosh(particle_kern * inv_width))**2
    
    # First partial derivatives
    dip_dD = inv_width * sech2_term / 4
    dip_dw = -particle_kern/2 * sech2_term
    
    # Second partial derivatives
    dip_DD = inv_width**2 / 4 * tanh_term * sech2_term
    dip_ww = particle_kern**2 * tanh_term * sech2_term
    dip_Dw = sech2_term * (1 - 2*inv_width * particle_kern * tanh_term) / 4
    
    # Gradient of "cost function"
    chi_D = residuals * dip_dD
    chi_w = residuals * dip_dw
    
    # Approximate Hessian of "cost function"
    chi_DD = dip_dD**2 + residuals * dip_DD
    chi_ww = dip_dw**2 + residuals * dip_ww
    chi_Dw = dip_dD * dip_dw + residuals * dip_Dw
    
    # Fill out approximate Hessian
    delta_arr = np.array([np.sum(chi_D), np.sum(chi_w)])
    hessian[0, 0] = np.sum(chi_DD)
    hessian[0, 1] = np.sum(chi_Dw)
    hessian[1, 0] = hessian[0, 1]
    hessian[1, 1] = np.sum(chi_ww)
    
    # Solve for parameter updates using pseudoinverse
    del_Dw = -delta_arr @ np.linalg.pinv(hessian)
    del_diameter = del_Dw[0]
    del_width = -inv_width * del_Dw[1] / (inv_width + del_Dw[1])
    
    return del_diameter, del_width

In [ ]:
# Find best diameter and width
ticker = 0
del_diameter = 1e99

# Optimize D and w
while (abs(del_diameter) > min_del_diameter) and (ticker < max_dw_ticker):
    
    # Run Newton's method for updated diameters and widths
    del_diameter, del_width = optimize_diameter_width(
        np.abs(peak_grids), 
        residuals, 
        diameter, 
        width
    )
    diameter = diameter + del_diameter
    width = width + del_width
    
    # Create kernels with new better diameters and widths, then find residuals
    peak_kernels = create_ideal_particle_image(np.abs(peak_grids), diameter, width)
    residuals = peak_kernels - data
    
    # Iterate
    print('.', end='', flush=True)
    ticker += 1

# Print optimization stats
print()  # newline
chi_squared = np.sum(residuals**2)
print(f'Chi-Squared={chi_squared}')

In [ ]:
def optimize_particle_positions(particle_grids, overlap, residuals, num_particles, diameter, width):
    """
    Calculate one Newton's step toward minimizing residuals^2 over particle centers.
    
    Parameters:
    -----------
    particle_grids : ndarray
        Complex grid (cx + i*cy) centered on each particle
    overlap : ndarray
        Image showing which particle owns each pixel
    residuals : ndarray
        Difference between model and actual image
    num_particles : int
        Number of particles
    diameter : float
        Particle diameter
    width : float
        Width parameter
    
    Returns:
    --------
    dx_positions : ndarray
        Change in x positions for each particle
    dy_positions : ndarray
        Change in y positions for each particle
        
    Notes:
    ------
    Uses Newton's method with Hessian to find position updates.
    Movement is limited to maxdr=20 pixels per step.
    
    Revision history:
    09/14/00 Mark D. Shattuck <mds> cidp2.m
    02/15/02 mds implement over and separate calc per particle
    06/25/04 mds limit change to maxdr=2
    04/30/07 mds changed meaning of w to 1/w
    """
    
    inv_width = 1 / width  # w is real width
    max_dr = 20  # max change per step
    
    hessian = np.zeros((2, 2))
    position_changes = np.zeros((num_particles, 2))
    
    # Create list of pixels for each particle
    flat_overlap = overlap.flatten()
    sorted_indices = np.argsort(flat_overlap)
    particle_counts = np.bincount(flat_overlap.astype(int), minlength=num_particles + 1)
    cumulative_counts = np.cumsum(particle_counts)
    
    # Useful numbers
    radial_distance = np.abs(particle_grids) + np.finfo(float).eps
    radial_distance_cubed = radial_distance**3 + np.finfo(float).eps
    x_component = np.real(particle_grids)
    y_component = np.imag(particle_grids)
    x_squared = x_component**2
    y_squared = y_component**2
    
    tanh_term = np.tanh((radial_distance - diameter/2) * inv_width)
    sech2_term = (1 / np.cosh((radial_distance - diameter/2) * inv_width))**2
    
    # First derivatives
    dip_dx = -inv_width * x_component * sech2_term / 2 / radial_distance
    dip_dy = -inv_width * y_component * sech2_term / 2 / radial_distance
    
    # Second derivatives
    dip_dxx = inv_width * sech2_term * (2*inv_width*x_squared*radial_distance*tanh_term - y_squared) / 2 / radial_distance_cubed
    dip_dyy = inv_width * sech2_term * (2*inv_width*y_squared*radial_distance*tanh_term - x_squared) / 2 / radial_distance_cubed
    dip_dxy = inv_width * x_component * y_component * sech2_term * (2*inv_width*radial_distance*tanh_term + 1) / 2 / radial_distance_cubed
    
    # Gradient components
    chi_x = residuals * dip_dx
    chi_y = residuals * dip_dy
    
    # Hessian components
    chi_xx = dip_dx**2 + residuals * dip_dxx
    chi_yy = dip_dy**2 + residuals * dip_dyy
    chi_xy = dip_dx * dip_dy + residuals * dip_dxy
    
    # Flatten arrays for indexing
    chi_x_flat = chi_x.flatten()
    chi_y_flat = chi_y.flatten()
    chi_xx_flat = chi_xx.flatten()
    chi_yy_flat = chi_yy.flatten()
    chi_xy_flat = chi_xy.flatten()
    
    # Loop over particles
    for particle_idx in range(num_particles):
        # Get indices for this particle's pixels
        start_idx = cumulative_counts[particle_idx]
        end_idx = cumulative_counts[particle_idx + 1]
        pixel_indices = sorted_indices[start_idx:end_idx]
        
        # Build gradient vector
        gradient = np.array([
            np.sum(chi_x_flat[pixel_indices]),
            np.sum(chi_y_flat[pixel_indices])
        ])
        
        # Build Hessian matrix
        hessian[0, 0] = np.sum(chi_xx_flat[pixel_indices])
        hessian[0, 1] = np.sum(chi_xy_flat[pixel_indices])
        hessian[1, 0] = hessian[0, 1]
        hessian[1, 1] = np.sum(chi_yy_flat[pixel_indices])
        
        # Newton's step
        position_changes[particle_idx, :] = gradient @ np.linalg.pinv(hessian)
    
    dx_positions = position_changes[:, 0]
    dy_positions = position_changes[:, 1]
    
    # Limit step size to max_dr
    position_change_magnitude = np.sqrt(dx_positions**2 + dy_positions**2)
    large_step_mask = position_change_magnitude > max_dr
    dx_positions[large_step_mask] = dx_positions[large_step_mask] / position_change_magnitude[large_step_mask]
    dy_positions[large_step_mask] = dy_positions[large_step_mask] / position_change_magnitude[large_step_mask]
    
    return dx_positions, dy_positions

In [ ]:
# Find best positions
ticker = 0
del_chi_squared = 1e99

# Optimize x, y positions
while (abs(del_chi_squared) > min_del_chi_squared) and (ticker < max_ticker):
    
    # Run Newton's method for updated positions
    dx_peak_loc, dy_peak_loc = optimize_particle_positions(
        peak_grids, 
        overlap_map, 
        residuals, 
        num_peaks, 
        diameter, 
        width
    )
    peak_x_locations = peak_x_locations + dx_peak_loc
    peak_y_locations = peak_y_locations + dy_peak_loc
    
    # Create new kernels and compute residuals
    peak_grids, overlap_map = create_particle_grid(
        peak_x_locations - half_kernel_size,
        peak_y_locations - half_kernel_size,
        grid_width,
        grid_height,
        [1, grid_width, 1, grid_height],
        num_peaks,
        2 * half_kernel_size + 3,
        0
    )
    peak_kernels = create_ideal_particle_image(peak_grids, diameter, width)
    residuals = peak_kernels - data
    
    # Calculate the error changes
    new_chi_squared = np.sum(residuals**2)
    del_chi_squared = chi_squared - new_chi_squared
    chi_squared = new_chi_squared
    
    # Iterate
    print('.', end='', flush=True)
    ticker += 1

# Print optimization stats
print()  # newline
chi_squared = np.sum(residuals**2)
print(f'Chi-Squared={chi_squared}')